# Plot Cold Past 
## Written by Eric Rohr

In [1]:
### import modules
import illustris_python as il
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.patheffects as pe
import matplotlib.transforms as transforms
from matplotlib.gridspec import GridSpec
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d
import scipy.stats
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import ks_2samp, anderson_ksamp
import os
import time
import h5py
import rohr_utils as ru 
import random
import six
%matplotlib widget

plt.style.use('fullpage.mplstyle')

zs, times = ru.return_zs_costimes()
times /= 1.0e9 # [Gyr]
scales = 1. / (1.+ zs)


In [ ]:
os.chdir('/u/reric/Scripts/')
! pwd

In [ ]:
def load_grpdict(infname):
    result = {}
    with h5py.File('../Output/TNG50-1_subfindGRP/' + infname, 'r') as f:
        for group_key in f.keys():
            result[group_key] = {}
            for dset_key in f[group_key].keys():
                result[group_key][dset_key] = f[group_key][dset_key][:]
        f.close()
    
    return result

infname = 'central_groups_subfind_TNG50-1_branches.hdf5'
groups_grp_dict = load_grpdict(infname)
halos_snapnum099_grp_dict = load_grpdict('central_subfind_TNG50-1_branches_snapNum099.hdf5')
halos_snapnum067_grp_dict = load_grpdict('central_subfind_TNG50-1_branches_snapNum067.hdf5')
halos_snapnum050_grp_dict = load_grpdict('central_subfind_TNG50-1_branches_snapNum050.hdf5')
halos_snapnum033_grp_dict = load_grpdict('central_subfind_TNG50-1_branches_snapNum033.hdf5')

grp_dicts = [groups_grp_dict,
             halos_snapnum099_grp_dict,
             halos_snapnum067_grp_dict,
             halos_snapnum050_grp_dict,
             halos_snapnum033_grp_dict]


In [ ]:
def compute_coldgasCGM(grp_dict):
    dset_key = 'SubhaloCGMColdGasMass'
    for group_key in grp_dict:
        group = grp_dict[group_key]
        group[dset_key] = np.zeros(group['radii'].shape[0], dtype=group['radii'].dtype) - 1
        for time_index in range(group['radii'].shape[0]):
            radii = group['radii'][time_index]
            r200c = group['HostGroup_R_Crit200'][time_index]
            subhalo_mcgas_shells = group['SubhaloColdGasMassShells'][time_index]
            mask = ((radii > r200c * 0.1) & (radii < r200c))
            subhalo_cgmcgmass = np.sum(subhalo_mcgas_shells[mask])
            group[dset_key][time_index] = subhalo_cgmcgmass
    return grp_dict

for grp_dict in grp_dicts:
    compute_coldgasCGM(grp_dict)

In [ ]:
grp_keys = ['SnapNum', 'CosmicTime', 'HostGroup_M_Crit200',
            'HostGroup_R_Crit200', 'SubhaloMass',
            'Subhalo_Mstar_Rgal', 'Subhalo_Rgal',
            'SubhaloColdGasMass', 'SubhaloHotGasMass', 'SubhaloGasMass',
            'SubhaloCGMColdGasMass']

def create_taudict(grp_dict, [snaps]):
    tauresult = {}
    # begin loop over subhalos
    for group_index, group_key in enumerate(grp_dict):
        group = result[group_key]
        # if just starting, then initialize the dictionary 
        if group_index == 0:
            tauresult['SubfindID'] = np.zeros(len(result_keys), dtype=int)
            tauresult['HostSubhaloGrNr'] = np.zeros(len(result_keys), dtype=int)
            for grp_key in grp_keys:
                for snap in snaps:
                    if 'quench' in snap:
                        tauresult_key = grp_key + '_snapNumquench'
                        tauresult[tauresult_key] = np.zeros(len(grp_dict),
                                                            dtype=group[grp_key].dtype) - 1
                    else:
                        tauresult_key = grp_key + '_snapNum%03d'%snap
                        tauresult[tauresult_key] = np.zeros(len(grp_dict),
                                                            dtype=group[grp_key].dtype) - 1
                    
        tauresult['SubfindID'][group_index] = group['SubfindID'][0]
        tauresult['HostSubhaloGrNr'][group_index] = group['HostSubhaloGrNr'][0]
        # finish initializing the the result
        # assign the values at z=0, which are always the 0th element in the array
        for grp_key in grp_keys:
            tauresult_key = grp_key + '_z0'
            tauresult[tauresult_key][group_index] = group[grp_key][0]
    # finish loop over the branches
    return tauresult
    

In [ ]:
groups_grp_dict_keys = list(groups_grp_dict.keys())

In [ ]:
np.log10(groups_grp_dict[groups_grp_dict_keys[0]]['SubhaloCGMColdGasMass'][0])

In [ ]:
grp_keys = ['SnapNum', 'CosmicTime', 'HostGroup_M_Crit200',
            'HostGroup_R_Crit200', 'SubhaloMass',
            'Subhalo_Mstar_Rgal', 'Subhalo_Rgal',
            'SubhaloColdGasMass', 'SubhaloHotGasMass', 'SubhaloGasMass',
            'SubhaloCGMColdGasMass']

def create_taudict(grp_dict, [snaps]):
    tauresult = {}
    # begin loop over subhalos
    for group_index, group_key in enumerate(grp_dict):
        group = result[group_key]
        # if just starting, then initialize the dictionary 
        if group_index == 0:
            tauresult['SubfindID'] = np.zeros(len(result_keys), dtype=int)
            tauresult['HostSubhaloGrNr'] = np.zeros(len(result_keys), dtype=int)
            for grp_key in grp_keys:
                for snap in snaps:
                    if 'quench' in snap:
                        tauresult_key = grp_key + '_snapNumquench'
                        tauresult[tauresult_key] = np.zeros(len(grp_dict),
                                                            dtype=group[grp_key].dtype) - 1
                    else:
                        tauresult_key = grp_key + '_snapNum%03d'%snap
                        tauresult[tauresult_key] = np.zeros(len(grp_dict),
                                                            dtype=group[grp_key].dtype) - 1
                    
        tauresult['SubfindID'][group_index] = group['SubfindID'][0]
        tauresult['HostSubhaloGrNr'][group_index] = group['HostSubhaloGrNr'][0]
        # finish initializing the the result
        # assign the values at z=0, which are always the 0th element in the array
        for grp_key in grp_keys:
            tauresult_key = grp_key + '_z0'
            tauresult[tauresult_key][group_index] = group[grp_key][0]
    # finish loop over the branches
    return tauresult
    

In [ ]:
group = groups_grp_dict[groups_grp_dict_keys[0]]
time_index = 99 == group['SnapNum']


In [ ]:
len(groups_grp_dict)